In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

%matplotlib inline

In [ ]:
client_id='7438769d2e044325a9871eae4000971a'
client_secret='e2b81e7957ae44a0862feb6e163d22fc'
redirect_uri='http://localhost:3000'

Since spotify API limits the number of tracks returned in a single query to 50 song track IDs, I will utilize a nested loop to retrieve all songs I've liked (1,094 songs) and append it to the same list

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

#Spotify API credentials
client_id='7438769d2e044325a9871eae4000971a'
client_secret='e2b81e7957ae44a0862feb6e163d22fc'  
redirect_uri = 'http://localhost:3000'       

# Setting up the Spotify OAuth handler
scope = 'user-library-read'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))


# create empty lists to store results
artist_name = []
track_name = []
popularity = []
track_id = []

for offset in range(0,1100,50):
    track_results = sp.current_user_saved_tracks(limit=50, offset=offset)
    for item in track_results['items']:
        track = item['track']
        artist_name.append(track['artists'][0]['name'])
        track_name.append(track['name'])
        track_id.append(track['id'])
        popularity.append(track['popularity'])

In [ ]:
len(track_id)

In [ ]:
import pandas as pd

df_tracks = pd.DataFrame({
    'artist_name': artist_name,
    'track_name': track_name,
    'popularity': popularity,
    'track_id': track_id
})

print(df_tracks.shape)
df_tracks.head()

Next, checking for duplicate songs. The same song can have different track_id's depending on whether it was a single release, part of album, etc

In [ ]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'], keep=False)].count()

In [ ]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace = True)

check to see if duplicates were dropped

In [ ]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

I also want the artist genre as this likely will be the biggest determinant of whether I will like a song or not. Some tracks may not have a genre, so I will check for this and assign the value of "unknown genre" if empty.

In [ ]:
client_id = '95220ed99e0d442eb4a05563cc60d01f' 
client_secret = 'c8489086c20148d580b51e7cb7298d87'    
redirect_uri = 'http://localhost:3000'       

scope = 'user-library-read'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))

track_ids = df_tracks['track_id'].tolist()

artist_ids = []
artist_genres = []

for track_id in track_ids:
    track_info = sp.track(track_id)
    artist_id = track_info['artists'][0]['id']
    artist_ids.append(artist_id)

    artist_info = sp.artist(artist_id)
    genres = artist_info['genres']
    artist_genres.append(genres)

df_tracks['artist_id'] = artist_ids
df_tracks['artist_genre'] = artist_genres

I noticed that some tracks have no genre, and just empty brackets []. I will replace these with "unknown genre". 

In [46]:
#check for empty lists in the 'artist_genre' column
has_empty_lists = df['artist_genre'].apply(lambda x: isinstance(x, list) and len(x) == 0).any()
has_empty_lists

False

In [47]:
def replace_empty_genre_list(genre):
    # Check if the genre is an empty list
    if isinstance(genre, list) and len(genre) == 0:
        return 'Unknown Genre'
    else:
        return genre

# Apply the function
df_tracks['artist_genre'] = df_tracks['artist_genre'].apply(replace_empty_genre_list)

# Verify
df_tracks[df_tracks['artist_genre'] == "Unknown Genre"].head()


NameError: name 'df_tracks' is not defined

Now we can retrive the audio features using the track ids

In [ ]:

client_id = '95220ed99e0d442eb4a05563cc60d01f' 
client_secret = 'c8489086c20148d580b51e7cb7298d87'    
redirect_uri = 'http://localhost:3000'       


scope = 'user-library-read'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))



all_audio_features = []

        
for i in range(0,len(df_tracks['track_id']),100):
    batch = df_tracks['track_id'][i:i+100]
    audio_features = sp.audio_features(batch)
    all_audio_features.extend(audio_features)
        

In [ ]:
df_audio_features = pd.DataFrame(all_audio_features)
df_audio_features.head()

Drop unecessary columns and change id to track_id to match first dataframe

In [ ]:
columns_to_drop = ['type', 'uri', 'track_href', 'analysis_url']

df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

In [ ]:
# Merge dataframes
df = pd.merge(df_tracks, df_audio_features, on='track_id',how='inner')

In [ ]:
df.to_csv('/Users/lindaseong/Desktop/Data_Science_Portfolio_work_2023/Project3_Spotify/spotify_liked_songs_metadata_audio_features_genres.csv')

In [58]:
df = pd.read_csv('/Users/lindaseong/Desktop/Data_Science_Portfolio_work_2023/Project3_Spotify/disliked_songs_metadata_genre_and_audio_features.csv', index_col = 0)

In [59]:
df

,artist_name,track_name,track_id,popularity,artist_id,artist_genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Rebecca Black,Friday,4fK6E2UywZTJIa5kWnCD6x,0,3Vl9fyKMIdLMswk8ai3mm9,Unknown Genre,0.684,0.889,8,-4.555,0,0.1020,0.030700,0.000003,0.7750,0.582,112.004,210026,4
1,Limp Bizkit,Rollin' (Air Raid Vehicle),3IV4swNduIRunHREK80owz,79,165ZgPlLkK7bf5bDoFc6Sb,"['alternative metal', 'funk metal', 'nu metal'...",0.603,0.933,1,-3.358,1,0.1710,0.005910,0.000000,0.2060,0.709,96.306,213760,4
2,Hones-T,Ass and Titties (feat. Qb5),2bCmpSXIQIVSvjgepbic3I,16,1gjYCdpkRbQf6hQmmGNPsj,Unknown Genre,0.705,0.785,9,-4.200,0,0.4030,0.027700,0.000004,0.1280,0.231,145.925,146387,4
3,Gabby Barrett,Glory Days,7qaMlLPdwNiiU1Uiy1oAdO,72,6Iz3eq2aQGFf7TbGT2iahL,['contemporary country'],0.647,0.776,7,-5.221,1,0.0514,0.006860,0.000000,0.1190,0.268,103.919,162733,4
4,Nate Smith,World on Fire,5ya0TmUQw2wHMkq36rPsnd,79,4NYMUsIcUUsBHbV9DICa5x,"['contemporary country', 'modern country pop']",0.461,0.887,11,-4.871,0,0.0429,0.000219,0.000000,0.1040,0.411,95.964,189987,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,Magnolia Park,Do Or Die,03hgA9bbWk17K66vN7wEzp,54,7B76SsfzG0wWk1WEvGzCmY,['pop punk'],0.653,0.795,6,-3.238,1,0.0679,0.342000,0.000104,0.0728,0.885,86.162,251491,4
925,ILLENIUM,Nothing Ever After,2foma3XC28Wk4Fcthqu7lk,59,45eNHdiiabvmbp4erw26rg,"['edm', 'melodic dubstep', 'pop dance']",0.537,0.964,9,-4.157,1,0.1130,0.000599,0.000011,0.0564,0.712,155.017,206903,4
926,BABYMETAL,BxMxC,4bbIpOYtDP48AdC3lzyqyE,31,630wzNP2OL7fl4Xl0GnMWq,"['idol rock', 'j-metal', 'kawaii metal']",0.490,0.956,5,-4.249,0,0.0675,0.000023,0.000000,0.4550,0.412,110.042,179187,4
927,thrown,grayout,6IZ7Pg6C3vduV2gQsycrYS,60,5eBCPtU2iPbzuMRre9BePt,Unknown Genre,0.411,0.939,1,-4.548,0,0.1190,0.000731,0.000000,0.9530,0.320,175.062,168887,4


In [60]:
#this code is for when loading from a csv file
# Extracting all genres from the 'artist_genre' column and removing brackets and quotes
all_genres = df['artist_genre'].str.replace('[\[\]\'\"]', '', regex=True).str.split(',').explode()

# Finding unique genres
unique_genres = list(set(all_genres))


len(unique_genres)

660

In [61]:
all_genres

0          Unknown Genre
1      alternative metal
1             funk metal
1               nu metal
1            post-grunge
             ...        
928    melodic metalcore
928            metalcore
928          modern rock
928                 rock
928         uk metalcore
Name: artist_genre, Length: 2426, dtype: object

How many unqiue genres are listed in the artist_genre column? The genres column is a list of several genres an artist is associated with. 

252 genres is quite a lot to handle, so I will assign the tracks to more broader genres. I will also include an unknown genre for the tracks that were not assigned artist genres. This process will also one-hot encode the data for our recommendation system.

In [62]:
genre_mappings = {
    'rock': 'Rock',
    'pop': 'Pop',
    'hip-hop': 'Hip-Hop/Rap',
    'rap': 'Hip-Hop/Rap',
    'jazz': 'Jazz',
    'classical': 'Classical',
    'r&b': 'R&B/Soul',
    'soul': 'R&B/Soul',
    'blues': 'Blues',
    'country': 'Country',
    'reggae': 'Reggae',
    'folk': 'Folk',
    'latin': 'Latin',
    'metal': 'Metal',
    'indie': 'Indie',
    'punk': 'Punk',
    'funk': 'Funk',
    'gospel': 'Gospel',
    'heavy metal': 'Heavy Metal',
    'k-pop': 'K-pop',
    'house': 'House/Techno',
    'techno': 'House/Techno',
    'trap': 'Trap/Dubstep',
    'dubstep': 'Trap/Dubstep',
    'electronic': 'Electronic/Dance',
    'dance': 'Electronic/Dance'
    # Additional mappings can be added here
}

In [49]:
df = df[:-6]
df

,artist_name,track_name,track_id,popularity,artist_id,artist_genre,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Rebecca Black,Friday,4fK6E2UywZTJIa5kWnCD6x,0.0,3Vl9fyKMIdLMswk8ai3mm9,[],0.684,0.889,8,-4.555,...,0.7750,0.5820,112.004,audio_features,4fK6E2UywZTJIa5kWnCD6x,spotify:track:4fK6E2UywZTJIa5kWnCD6x,https://api.spotify.com/v1/tracks/4fK6E2UywZTJ...,https://api.spotify.com/v1/audio-analysis/4fK6...,210026,4
1,Limp Bizkit,Rollin' (Air Raid Vehicle),3IV4swNduIRunHREK80owz,79.0,165ZgPlLkK7bf5bDoFc6Sb,"['alternative metal', 'funk metal', 'nu metal'...",0.603,0.933,1,-3.358,...,0.2060,0.7090,96.306,audio_features,3IV4swNduIRunHREK80owz,spotify:track:3IV4swNduIRunHREK80owz,https://api.spotify.com/v1/tracks/3IV4swNduIRu...,https://api.spotify.com/v1/audio-analysis/3IV4...,213760,4
2,Hones-T,Ass and Titties (feat. Qb5),2bCmpSXIQIVSvjgepbic3I,16.0,1gjYCdpkRbQf6hQmmGNPsj,[],0.705,0.785,9,-4.200,...,0.1280,0.2310,145.925,audio_features,2bCmpSXIQIVSvjgepbic3I,spotify:track:2bCmpSXIQIVSvjgepbic3I,https://api.spotify.com/v1/tracks/2bCmpSXIQIVS...,https://api.spotify.com/v1/audio-analysis/2bCm...,146387,4
3,Gabby Barrett,Glory Days,7qaMlLPdwNiiU1Uiy1oAdO,72.0,6Iz3eq2aQGFf7TbGT2iahL,['contemporary country'],0.647,0.776,7,-5.221,...,0.1190,0.2680,103.919,audio_features,7qaMlLPdwNiiU1Uiy1oAdO,spotify:track:7qaMlLPdwNiiU1Uiy1oAdO,https://api.spotify.com/v1/tracks/7qaMlLPdwNii...,https://api.spotify.com/v1/audio-analysis/7qaM...,162733,4
4,Nate Smith,World on Fire,5ya0TmUQw2wHMkq36rPsnd,79.0,4NYMUsIcUUsBHbV9DICa5x,"['contemporary country', 'modern country pop']",0.461,0.887,11,-4.871,...,0.1040,0.4110,95.964,audio_features,5ya0TmUQw2wHMkq36rPsnd,spotify:track:5ya0TmUQw2wHMkq36rPsnd,https://api.spotify.com/v1/tracks/5ya0TmUQw2wH...,https://api.spotify.com/v1/audio-analysis/5ya0...,189987,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,As I Lay Dying,My Own Grave,0CcqWuAEJC93K8cBMbAjgI,63.0,2vd2HnNh4pdYa9gDVHFjEu,"['alternative metal', 'christian metal', 'melo...",0.477,0.994,5,-4.953,...,0.0534,0.0397,125.058,audio_features,0CcqWuAEJC93K8cBMbAjgI,spotify:track:0CcqWuAEJC93K8cBMbAjgI,https://api.spotify.com/v1/tracks/0CcqWuAEJC93...,https://api.spotify.com/v1/audio-analysis/0Ccq...,253318,4
915,Limp Bizkit,My Way,2gSVKxPDww9Eep5rdvtdem,75.0,165ZgPlLkK7bf5bDoFc6Sb,"['alternative metal', 'funk metal', 'nu metal'...",0.576,0.864,6,-3.981,...,0.6830,0.6090,97.225,audio_features,2gSVKxPDww9Eep5rdvtdem,spotify:track:2gSVKxPDww9Eep5rdvtdem,https://api.spotify.com/v1/tracks/2gSVKxPDww9E...,https://api.spotify.com/v1/audio-analysis/2gSV...,272973,4
916,Spiritbox,Jaded,6IdyYbGg1jxiWhfwm2Ykjn,67.0,4MzJMcHQBl9SIYSjwWn8QW,['canadian metalcore'],0.320,0.953,11,-4.119,...,0.0857,0.1430,85.430,audio_features,6IdyYbGg1jxiWhfwm2Ykjn,spotify:track:6IdyYbGg1jxiWhfwm2Ykjn,https://api.spotify.com/v1/tracks/6IdyYbGg1jxi...,https://api.spotify.com/v1/audio-analysis/6Idy...,262632,4
917,Bring Me The Horizon,Kingslayer (feat. BABYMETAL),7CAbF0By0Fpnbiu6Xn5ZF7,76.0,1Ffb6ejR6Fe5IamqA5oRUF,"['melodic metalcore', 'metalcore', 'modern roc...",0.434,0.929,1,-2.749,...,0.0916,0.4280,92.541,audio_features,7CAbF0By0Fpnbiu6Xn5ZF7,spotify:track:7CAbF0By0Fpnbiu6Xn5ZF7,https://api.spotify.com/v1/tracks/7CAbF0By0Fpn...,https://api.spotify.com/v1/audio-analysis/7CAb...,220042,4


In [63]:
# Modify the map_genres function to handle non-list entries in 'artist_genre' column

def map_genres(row, genre_mappings):
    try:
        # Attempt to convert string representation of list to actual list
        subgenres = eval(row['artist_genre'])
    except:
        # If conversion fails, treat it as an empty list
        subgenres = []
    
    broader_genres = set()
    for subgenre in subgenres:
        for key, value in genre_mappings.items():
            if key in subgenre.lower():
                broader_genres.add(value)
    
    # Add 'Unknown Genre' if no broader genres are found
    if not broader_genres:
        broader_genres.add('Unknown Genre')
    
    return list(broader_genres)

# Reapply the mapping function
df['broader_genres'] = df.apply(lambda row: map_genres(row, genre_mappings), axis=1)

# Update the list of all broader genres (to include 'Unknown Genre')
all_broader_genres = list(set(genre_mappings.values())) + ['Unknown Genre']

# Reinitialize columns for each broader genre with 0
for genre in all_broader_genres:
    df[genre] = 0

# Reset 1 for the presence of each broader genre
for index, row in df.iterrows():
    for genre in row['broader_genres']:
        df.at[index, genre] = 1

# Display the updated dataframe
df


,artist_name,track_name,track_id,popularity,artist_id,artist_genre,danceability,energy,key,loudness,...,Punk,Reggae,Jazz,Rock,Indie,Metal,Electronic/Dance,Blues,House/Techno,Unknown Genre
0,Rebecca Black,Friday,4fK6E2UywZTJIa5kWnCD6x,0,3Vl9fyKMIdLMswk8ai3mm9,Unknown Genre,0.684,0.889,8,-4.555,...,0,0,0,0,0,0,0,0,0,1
1,Limp Bizkit,Rollin' (Air Raid Vehicle),3IV4swNduIRunHREK80owz,79,165ZgPlLkK7bf5bDoFc6Sb,"['alternative metal', 'funk metal', 'nu metal'...",0.603,0.933,1,-3.358,...,0,0,0,1,0,1,0,0,0,0
2,Hones-T,Ass and Titties (feat. Qb5),2bCmpSXIQIVSvjgepbic3I,16,1gjYCdpkRbQf6hQmmGNPsj,Unknown Genre,0.705,0.785,9,-4.200,...,0,0,0,0,0,0,0,0,0,1
3,Gabby Barrett,Glory Days,7qaMlLPdwNiiU1Uiy1oAdO,72,6Iz3eq2aQGFf7TbGT2iahL,['contemporary country'],0.647,0.776,7,-5.221,...,0,0,0,0,0,0,0,0,0,0
4,Nate Smith,World on Fire,5ya0TmUQw2wHMkq36rPsnd,79,4NYMUsIcUUsBHbV9DICa5x,"['contemporary country', 'modern country pop']",0.461,0.887,11,-4.871,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,Magnolia Park,Do Or Die,03hgA9bbWk17K66vN7wEzp,54,7B76SsfzG0wWk1WEvGzCmY,['pop punk'],0.653,0.795,6,-3.238,...,1,0,0,0,0,0,0,0,0,0
925,ILLENIUM,Nothing Ever After,2foma3XC28Wk4Fcthqu7lk,59,45eNHdiiabvmbp4erw26rg,"['edm', 'melodic dubstep', 'pop dance']",0.537,0.964,9,-4.157,...,0,0,0,0,0,0,1,0,0,0
926,BABYMETAL,BxMxC,4bbIpOYtDP48AdC3lzyqyE,31,630wzNP2OL7fl4Xl0GnMWq,"['idol rock', 'j-metal', 'kawaii metal']",0.490,0.956,5,-4.249,...,0,0,0,1,0,1,0,0,0,0
927,thrown,grayout,6IZ7Pg6C3vduV2gQsycrYS,60,5eBCPtU2iPbzuMRre9BePt,Unknown Genre,0.411,0.939,1,-4.548,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Extracting the broad genre columns into a new dataframe
broader_genres_df = df.iloc[:, -22:]

In [16]:
df.to_csv('/Users/lindaseong/Desktop/Data_Science_Portfolio_work_2023/Project3_Spotify/disliked_songs_metadata_and_audio_features_w_broad_genre_categories.csv')

Now I'm going to repeat the spotify scraping and data processing for my disliked songs playlist!
The only difference here is that i need to obtain the spotify ID of my disliked playlist using the url link and then pull the songs from that specific ID. For playlists, we can pull 100 songs at a time. I will also include a check to avoid errors for missing or invalid track data. For some reason, I had to modify the code to directly assign danceability and other audio features, in order to avoid NaN for danceability on all tracks.

In [17]:
client_id = '95220ed99e0d442eb4a05563cc60d01f' 
client_secret = 'c8489086c20148d580b51e7cb7298d87'    
redirect_uri = 'http://localhost:3000'
playlist_id = '6nPX2DGoVNJvO9CIjhdBDu'


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope="playlist-read-private"))


artist_name = []
track_name = []
popularity = []
track_id = []
artist_id = []

for offset in range(0,950,100):
    track_results = sp.playlist_tracks(playlist_id, limit=100, offset=offset)
    for item in track_results['items']:
        track = item['track']
        artist_name.append(track['artists'][0]['name'])
        track_name.append(track['name'])
        track_id.append(track['id'])
        popularity.append(track['popularity'])
        artist_id.append(track['artists'][0]['id'])

dislikes_df_tracks = pd.DataFrame({
    'artist_name': artist_name,
    'track_name': track_name,
    'track_id': track_id,
    'popularity': popularity,
    'artist_id': artist_id,
    
})

print(dislikes_df_tracks.shape)
dislikes_df_tracks

NameError: name 'spotipy' is not defined

In [ ]:
client_id = '95220ed99e0d442eb4a05563cc60d01f' 
client_secret = 'c8489086c20148d580b51e7cb7298d87'    
redirect_uri = 'http://localhost:3000'       
playlist_id = '6nPX2DGoVNJvO9CIjhdBDu'


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope="playlist-read-private"))

artist_ids = dislikes_df_tracks['artist_id'].tolist()

artist_genres = []

for artist_id in artist_ids:
    artist_info = sp.artist(artist_id)
    genres = artist_info.get('genres')
    artist_genres.append(genres)

dislikes_df_tracks['artist_genre'] = artist_genres

In [ ]:
dislikes_df_tracks

In [ ]:
dislike_audio_features = []

for i in range(0,len(dislikes_df_tracks['track_id']),100):
    batch = dislikes_df_tracks['track_id'][i:i+100]
    audio_features = sp.audio_features(batch)
    dislike_audio_features.extend(audio_features)

dislikes_audio_features_df = pd.DataFrame(dislike_audio_features)
dislikes_playlist_df = pd.concat([dislikes_df_tracks, dislikes_audio_features_df], axis=1)
dislikes_playlist_df.head()

In [ ]:
columns_to_drop = ['type', 'uri', 'id', 'track_href', 'analysis_url']

dislikes_playlist_df.drop(columns_to_drop, axis=1,inplace=True)

dislikes_playlist_df.shape

In [ ]:
dislikes_playlist_df.to_csv("/Users/lindaseong/Desktop/Data_Science_Portfolio_work_2023/Project3_Spotify/disliked_songs_metadata_genre_and_audio_features.csv")

In [ ]:
#check for empty lists in the 'artist_genre' column
has_empty_lists = dislikes_playlist_df['artist_genre'].apply(lambda x: isinstance(x, list) and len(x) == 0).any()
has_empty_lists

In [ ]:
# Apply the replace function 
dislikes_playlist_df['artist_genre'] = dislikes_playlist_df['artist_genre'].apply(replace_empty_genre_list)

# Verify if the replacement worked
dislikes_playlist_df[dislikes_playlist_df['artist_genre'] == "Unknown Genre"].head()

In [ ]:
all_genres = []
for item in dislikes_playlist_df['artist_genre']:
    if isinstance(item, list):
        all_genres.extend(item)
    elif isinstance(item, str):
        all_genres.append(item)

unique_genres = list(set(all_genres))

# Apply the adjusted mapping function to get the unique genres
broader_categories = map_to_broader_categories(unique_genres)

# Create a DataFrame for broader categories based on the presence of specific genres in each song
for broader_category in broader_categories.keys():
    dislikes_playlist_df[broader_category] = dislikes_playlist_df['artist_genre'].apply(lambda genres: any(genre in broader_categories[broader_category] for genre in genres)).astype(int)

# Extracting the broad genre columns into a new dataframe
dislikes_broader_genres_df = dislikes_playlist_df.iloc[:, -22:]

In [ ]:
dislikes_playlist_df.to_csv("/Users/lindaseong/Desktop/Data_Science_Portfolio_work_2023/Project3_Spotify/disliked_songs_metadata_and_audio_features_w_broad_genre_categories.csv")

Next i'm going to add a column to denote whether the song is from my likes or dislikes and combine the dataframes for EDA. 

In [ ]:
# Add a 'preference' column to each dataset
df['preference'] = 'like'
dislikes_playlist_df['preference'] = 'dislike'

all_songs_df = pd.concat([df, dislikes_playlist_df], ignore_index=True)

all_songs_df

In [ ]:
all_songs_df.to_csv("/Users/lindaseong/Desktop/Data_Science_Portfolio_work_2023/Project3_Spotify/all_songs_metadata_and_audio_features_w_broad_genres.csv")

## Exploratory Data Analysis


The data features have the following definitions according to spotify's API documentation:

**acousticness** : A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

**danceability** : Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

**duration_ms** : The duration of the track in milliseconds.

**energy** : Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

**instrumentalness** : Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
key : The key the track is in. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on.

**liveness** : Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.

**loudness** : The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.

**mode** : Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.

**speechiness** : Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

**tempo** : The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

**time_signature** : An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure).

**valence** : A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

First, we can look at how popular the songs i've liked are, and which songs are the most popular in my liked list. Popularity is determined by how many times a song was played

In [ ]:
all_df = pd.read_csv('/Users/lindaseong/Desktop/Data_Science_Portfolio_work_2023/Project3_Spotify/all_songs_metadata_and_audio_features_w_broad_genres.csv', index_col=0)

I will start with identifying trends of my liked songs but I want to first see which songs are the most popular. 

In [ ]:
# create new dataframe consisting of top 100 most popular tracks
all_df_top = all_df.sort_values('popularity', ascending=False).head(100)

# show the first 50 results
all_df_top[['artist_name', 'track_name', 'popularity', 'preference']].head(10)

Most popular songs are liked, #2 is disliked by me. Ok, moving on to liked songs only.

In [ ]:
#show top 10 artists by number of tracks that appear in my top 100 most popular liked songs
df_top = df.sort_values('popularity', ascending=False).head(100)

df_top[['artist_name','track_name']].groupby('artist_name').count().sort_values('track_name', ascending=False).head(10)


In [ ]:
#show top 10 artists by total popularity that appear in my top 100 most popular liked songs

df_top[['artist_name','popularity']].groupby('artist_name').sum().sort_values('popularity', ascending=False).head(10)


Next, I want to determine whether popular songs are correlated with song attributes. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only the columns with audio features we want to analyze and popularity. 
#Will start with all attributes to get an overall sense of the data
features = ['popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

df_features = df_top[features]

# Generate the correlation matrix
corr_matrix = df_features.corr()


In [ ]:
# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Matrix of Audio Features and Popularity")
plt.show()


It appears that there's no real coorelation with a song's audio features and its popularity. There's a slight positive correlation between popularity and mode, instrumentalness, and tempo. We can also see that valence, which is how positive a song is, is positively correlated to danceability, energy, and loudness. 

Popularity is a measure from the entire population, and may not be an accurate reflection of what type of songs I enjoy. I now want to analyze the song features from all my liked songs to determine if I gravitate towards a certain style. 

First, I will plot the audio features to see the data distribution

In [ ]:
df.describe()

For popularity, I can tell that most songs I've liked are not as popular, given that the 75th percentile is at 62 of 100. The mean value for danceability and energy are high at around .7 and .73,respectively, and the 25th percentile for both are at .61 and .62, respectively, so it's clear that I generally prefer to listen to songs with high energy and a good, rhythmic beat.

In [ ]:
#generate histograms to get a sense of distribution for each feature

# List of audio features to analyze
features = ['danceability', 'energy', 'loudness', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Plot histograms for each feature
plt.figure(figsize=(20, 10))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i + 1)  # 3x3 grid of subplots
    sns.histplot(df_audio_features[feature], kde=True)
    plt.title(f'Distribution of {feature.capitalize()}')
    plt.xlabel(feature.capitalize())

plt.tight_layout()
plt.show()

From the histograms, I can see that danceability and energy have a somewhat normal distrubtion but also slightly left skewed, indicating I like more danceable and energetic tracks. Loudness has a slight left-skewed tail but otherwise normal distribution, and shows that most of the songs I enjoy listen to are not that loud (given the scale goes to -60dB). Speechiness, acousticness, instrumentalness, and liveness are all heavily right-skewed, indicating that most tracks have less words, are not live, and are not acououstic or instrumental in nature. Valence and tempo has a normal distribution, indicating a diverse range in positivity and tempo in the tracks. 

Next, I will use blox plots to visualize median, quantiles, and any outliers. 

In [ ]:
# Plot box plots for each feature
    
plt.figure(figsize=(15, 15))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i + 1)  
    sns.boxplot(x=df_audio_features[feature])
    plt.title(f'Box Plot of {feature.capitalize()}')
    plt.xlabel(feature.capitalize())

plt.tight_layout()
plt.show()

Boxplots further support our results, with danceability, energy, and loudness being left-skewed, vlaence and tempo somewhat normal, and rest being left-skewed. 

Last part of the exploratory analysis is to check the correlation of the features

In [ ]:
# Calculate the correlation matrix
corr_matrix = df_audio_features[features].corr()

# Plot the heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Matrix of Audio Features")
plt.show()

Energy and Loudness have a strong positive correlation, which is expected as lounder tracks are perceived as more energetic. Similarly, energy has a slight positive correlation with tempo, suggesting that faster songs are more energetic. Energy has a strong negative correlation with acousticness, suggesting tracks with higher acousticness tend to have lower energy. Likewise, acousticness and loudness have a negative correlation, indicating acoustic tracks are generally quieter. Valence shows a moderately positive correlation with danceability and slight positive correlation with energy, indicating that the tracks I like that are more positive are often more danceable and energetic. Tempo is slightly negatively correlated with acousticness and danceability, suggesting that acoustic songs have a slower tempo, and that tracks with slower tempos are better suitable for dancing. 

All in all, these correlation relationships are not too suprising and are somewhat intuitive. 

In [ ]:
# Extracting danceability and valence
X = df_audio_features['danceability'].values # Predictor variable
y = df_audio_features['valence'].values  # Response variable

X = X.reshape(X.shape[0], 1)
y = y.reshape(y.shape[0], 1)

# Create a linear regression model
model = linear_model.LinearRegression()
model.fit(X, y)

# Make predictions
y_pred = model.predict(X)

# Calculate R^2 value
r_squared = model.score(X, y)

# Scatterplot with fitted line
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_audio_features['danceability'], y=df_audio_features['valence'])
plt.plot(df_audio_features['danceability'], y_pred, color='red')  # Fitted line
plt.title('Correlation between Danceability and Song Mood')
plt.xlabel('Danceability')
plt.ylabel('Valence')
plt.text(0.05, 0.95, f'R² = {r_squared:.2f}', transform=plt.gca().transAxes)  # Display R² value on the plot

# Adding smaller ticks
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.02))

ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.02))

plt.show()

In [ ]:
# Extracting danceability and valence
X = df_audio_features['energy'].values # Predictor variable
y = df_audio_features['valence'].values  # Response variable

X = X.reshape(X.shape[0], 1)
y = y.reshape(y.shape[0], 1)

# Create a linear regression model
model = linear_model.LinearRegression()
model.fit(X, y)

# Make predictions
y_pred = model.predict(X)

# Calculate R^2 value
r_squared = model.score(X, y)

# Scatterplot with fitted line
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_audio_features['energy'], y=df_audio_features['valence'])
plt.plot(df_audio_features['energy'], y_pred, color='red')  # Fitted line
plt.title('Correlation between Energy and Song Mood')
plt.xlabel('Energy')
plt.ylabel('Valence')
plt.text(0.05, 0.95, f'R² = {r_squared:.2f}', transform=plt.gca().transAxes)  # Display R² value on the plot

# Adding smaller ticks
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.02))

ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.02))

plt.show()

Lastly, I want to look at the distribution of genres within my liked songs to see which are my top broader categories.

In [ ]:
# Summing up the count for each broader genre category
genre_counts = broader_genres_df.sum().sort_values(ascending=False)

# Create a countplot with a valid color palette
plt.figure(figsize=(12, 8))
sns.barplot(x=genre_counts.values, y=genre_counts.index, palette="viridis")
plt.title("Count of Songs in Each Broader Genre Category")
plt.xlabel("Number of Songs")
plt.ylabel("Broader Genres")
plt.show()

I next want to see if any of these categories are correlated with audio features.

In [ ]:
# Assuming the original data (without 'Genres' and 'Unnamed: 0' columns) is stored in original_data_processed
audio_features = df_audio_features.select_dtypes(include=['float64', 'int64'])

# Merge the audio features with the one-hot encoded genre data
audio_genres_merged = pd.concat([audio_features, broader_genres_df], axis=1)

# Calculate the correlation matrix
correlation_matrix = audio_genres_merged.corr()

# Extract the correlations between genres and audio features
# We are interested in the last 'len(broader_genres_df.columns)' rows, which correspond to the genres
genre_audio_correlations = correlation_matrix.iloc[-len(broader_genres_df.columns):, :]

# Display the correlations for review
genre_audio_correlations.head()  # Displaying only first few rows for brevity


In [ ]:
# Removing rows and columns with NaN values (genres with insufficient data) for clearer visualization
genre_audio_correlations_cleaned = genre_audio_correlations.dropna(axis=0, how='all').dropna(axis=1, how='all')

# Plotting the cleaned correlation heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(genre_audio_correlations_cleaned, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Heatmap between Broader Genres and Audio Features (Cleaned)")
plt.show()


Popularity: 'Hip-Hop/Rap' and 'K-pop' show a positive correlation, suggesting these genres are associated with more popular songs in your dataset.

Danceability: Genres like 'Hip-Hop/Rap' and 'Latin' have a positive correlation with danceability, indicating these genres tend to feature more danceable tracks.

Energy: 'House/Techno' and 'Dance' genres show a strong positive correlation with energy, which is typical for these high-tempo, upbeat genres.

Loudness: Similarly, 'House/Techno' and 'Dance' genres have a positive correlation with loudness.

Speechiness: 'Hip-Hop/Rap' has a significantly high positive correlation with speechiness, which aligns with the vocal nature of rap music.

Now moving on to disliked songs

In [ ]:
#generate histograms to get a sense of distribution for each feature

# List of audio features to analyze
features = ['danceability', 'energy', 'loudness', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Plot histograms for each feature
plt.figure(figsize=(20, 10))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i + 1)  # 3x3 grid of subplots
    sns.histplot(dislikes_playlist_df[feature], kde=True)
    plt.title(f'Distribution of {feature.capitalize()}')
    plt.xlabel(feature.capitalize())

plt.tight_layout()
plt.show()

In [ ]:
# Plot box plots for each feature
    
plt.figure(figsize=(15, 15))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i + 1)  
    sns.boxplot(x=dislikes_playlist_df[feature])
    plt.title(f'Box Plot of {feature.capitalize()}')
    plt.xlabel(feature.capitalize())

plt.tight_layout()
plt.show()

In [ ]:
# Calculate the correlation matrix
corr_matrix = dislikes_playlist_df[features].corr()

# Plot the heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Matrix of Audio Features")
plt.show()